# AutoML classification with SDK v2
## Memo
- Didn't register data into Azure ML WS, just used Input()
## Progress
入力データセットのinput指定方法がよくわかっていない。サンプルノートブックだとフォルダまでしか指定してない？だとするとどうやってデータセットを指定？？

In [4]:
# test
## install Python SDK v2
# !pip install --pre azure-ai-ml
## check SDK version
# !pip show azure-ai-ml
## upgrade if necessary 
# !pip install --pre --upgrade azure-ai-ml

Name: azure-ai-ml
Version: 0.1.0b3
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuremlsdk@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: docker, applicationinsights, tqdm, marshmallow, msrest, isodate, azure-storage-blob, azure-identity, azure-common, pathspec, azure-storage-file-share, colorama, azure-core, jsonschema, pydash, typing-extensions, pyjwt, pyyaml, azure-mgmt-core
Required-by: 


In [2]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
output["Storage Account"] = workspace.storage_account.split("/")[-1]
output

Found the config file in: ./config.json


{'Workspace': 'ml-lab',
 'Subscription ID': 'f57ce3c6-5c6f-4f1e-8cba-b782d8974590',
 'Resource Group': 'rg-aml',
 'Location': 'westus2',
 'Storage Account': 'mllab6442967193'}

In [3]:
# Compute resources
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, 
        size="Standard_DS3_v2", 
        min_instances=0, 
        max_instances=4,
        tier="LowPriority",
        idle_time_before_scale_down=300
    )
    ml_client.compute.begin_create_or_update(compute)

In [6]:
# Prepare data
from azure.ai.ml import Input
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Create MLTables for training dataset
# 冗長かも。
try:
    registered_data_asset = ml_client.data.get(name='titanic-mltable-train', version="1")
    my_training_data_input = Input(
    type=AssetTypes.MLTABLE, 
    path=registered_data_asset.id
    )
except Exception:
    my_data = Data(
        type=AssetTypes.MLTABLE, 
        path="./data/training",
        name="titanic-mltable-train",
        description="Titanic data for training, created by SDK v2",
    )

    ml_client.data.create_or_update(my_data)

    registered_data_asset = ml_client.data.get(name='titanic-mltable-train') #もう一回名前ベースで取得しなくても、idゲットするメソッドありそう
    my_training_data_input = Input(
    type=AssetTypes.MLTABLE, 
    path=registered_data_asset.id
    )


In [7]:
# Configure AutoML classification job
from azure.ai.ml import automl

# General job parameters
compute_name = cpu_compute_target
max_trials = 5
exp_name = "dpv2_classification_titanic"

# Create the AutoML classification job with the related factory-function.

classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="Survived",
    primary_metric="AUC_weighted",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"dpv2": "SDKv2"},
)

# Limits are all optional
classification_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

# # Training properties are optional
# classification_job.set_training(
#     blocked_training_algorithms=["LogisticRegression"],
#     enable_onnx_compatible_models=True,
# )

Class ClassificationJob: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [8]:
# Submit the AutoML job (CDLTLL: Is it ml_client.create_or_update(classification_job))
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

Created job: ClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'task_type': <TaskType.CLASSIFICATION: 'Classification'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'heroic_shark_g9qwwvcqd1', 'description': None, 'tags': {'dpv2': 'SDKv2'}, 'properties': {}, 'id': '/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourceGroups/rg-aml/providers/Microsoft.MachineLearningServices/workspaces/ml-lab/jobs/heroic_shark_g9qwwvcqd1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData object at 0x7f783c030430>, 'serialize': <msrest.serialization.Serializer object at 0x7f783c1d4b50>, 'inputs': {}, 'display_name': 'heroic_shark_g9qwwvcqd1', 'experiment_name': 'dpv2_classification_titanic', 'compute': 'cpu-cluster', 'services': {'Tracking': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.JobService object at 0

'https://ml.azure.com/runs/heroic_shark_g9qwwvcqd1?wsid=/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourcegroups/rg-aml/workspaces/ml-lab&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'

In [9]:
ml_client.jobs.stream(returned_job.name)
print(returned_job.name)

RunId: heroic_shark_g9qwwvcqd1
Web View: https://ml.azure.com/runs/heroic_shark_g9qwwvcqd1?wsid=/subscriptions/f57ce3c6-5c6f-4f1e-8cba-b782d8974590/resourcegroups/rg-aml/workspaces/ml-lab


In [10]:
## for debug
# for x in dir(workspace):
#     print(x, ':', type(eval("workspace."+x)))

# Refs
## Docs
[SDK v2 (プレビュー) を使ってデータを操作する - Azure Machine Learning | Microsoft Docs](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-use-data?tabs=use-local%2Crw-blob)

## Notebooks
[SDK v2 exampless](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk)  
[Workspace configuration](https://github.com/Azure/azureml-examples/blob/sdk-preview/sdk/jobs/configuration.ipynb)  
[Training](https://github.com/Azure/azureml-examples/blob/sdk-preview/sdk/jobs/automl-standalone-jobs/automl-classification-task-bankmarketing/automl-classification-task-bankmarketing.ipynb)  
[MLTable](https://github.com/Azure/azureml-examples/blob/samuel100/mltable/sdk/assets/data/working_with_mltable.ipynb)  

## SDK v2 references
[azure.ai.ml package - Azure Machine Learning Python | Microsoft Docs](https://docs.microsoft.com/ja-jp/python/api/azure-ai-ml/azure.ai.ml?view=azure-ml-py)  
[azure.ai.ml.entities.AmlCompute class - Azure Machine Learning Python | Microsoft Docs](https://docs.microsoft.com/ja-jp/python/api/azure-ai-ml/azure.ai.ml.entities.amlcompute?view=azure-ml-py)  